<a href="https://colab.research.google.com/github/hy-JuHyeongkim/hansabong2/blob/main/hansabong_manual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/hy-JuHyeongkim/hansabong2.git
%cd /content/hansabong2/yolov5
%pip install -r requirements.txt
!pip install PyMuPDF # requirements 에 추가

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 구글드라이브 -> 공유문서함 에 가서
## 여기서 쓰고싶은 폴더에 우클릭>정리>바로가기 추가 하면 됩니다

In [ ]:
import fitz  # PyMuPDF
from PIL import Image
import os

# PDF 파일 경로 설정
pdf_path = '/content/drive/MyDrive/연구실서고 스캔/2024-여름/민정_이성산성/3-4.pdf'  # 본인의 PDF 파일 경로로 변경
output_folder = '/content/drive/MyDrive/연구실서고 스캔/2024-여름/주형_안면도/1차/test'    # 이미지 파일을 저장할 폴더 경로로 변경

# 출력 폴더가 존재하지 않으면 생성
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# PDF 파일 열기
pdf_document = fitz.open(pdf_path)

img_list = []
# 각 페이지를 이미지로 변환
for page_num in range(len(pdf_document)):
    page = pdf_document.load_page(page_num)
    pix = page.get_pixmap()
    img_path = os.path.join(output_folder, f'{page_num + 1}.png')
    img_list.append(img_path)

    pix.save(img_path)

print(f"PDF 파일이 성공적으로 이미지로 변환되었습니다. 이미지는 {output_folder}에 저장되었습니다.")

In [ ]:
import subprocess

page_photo_counts_list = [5, 5, 5, 5, 5, 5, 4, 5, 5, 2, 5, 3, 5, 5] #각 페이지 별 몇 장의 사진이 있는지 리스트 타입으로 입력해주세요
custom_save_path = "/content/drive/MyDrive/연구실서고 스캔/2024-여름/주형_안면도/1차/test_result" # 사진들이 저장될 경로를 지정해주세요
custom_name_prefix = "안면도 칼라 1차" # 저장될 사진들의 이름을 입력해주세요 (ex. abc 로 입력할 경우 사진들이 abc_1, abc_2, ... 이렇게 저장됩니다)
i=0 #시작 사진번호를 알려주세요 (ex 세 번째 pdf 를 작업하게 될 경우, 첫 번째 앨범의 총 사진 수 + 두 번째 앨범의 총 사진 수)
for img_path,page_photo_counts in zip(img_list, page_photo_counts_list):

    command = [
        "python", "/content/hansabong2/yolov5/detect.py",
        "--img", "416",
        "--weights", "/content/hansabong2/yolov5/runs/train/picture_yolo5m_results/weights/best.pt",
        "--source", img_path,
        "--save-crop",
        "--custom-save-path", custom_save_path,
        "--custom-name-prefix", custom_name_prefix,
        "--page-photo-counts", str(page_photo_counts),
        "--start-number", str(i+1)
    ]

    result = subprocess.run(command, capture_output=True, text=True)
    print(result.stdout)
    print(result.stderr)
    i+=page_photo_counts
